In [53]:
#!pip install fasttext
#!pip install transformers
#import nltk
#nltk.download('punkt')
#!pip3 install torch torchvision torchaudio

In [3]:
DATA_DIR = "./data/"
VALID_SIZE = .2
MODEL_PATH = "model_task2.bin"

In [4]:
import pandas as pd
import numpy as np
import scipy
import nltk
import re
import os
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from torch.optim import AdamW
from fasttext import load_model
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from tqdm import tqdm
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import preprocessing, model_selection
from torch import LongTensor
from sklearn.metrics import f1_score, confusion_matrix, classification_report

---

In [5]:
df = pd.read_csv('../Dataset/data/restaurants_laptop_train_with_pos_task2_cleaned.csv')

In [6]:
df.polarity = df.polarity.replace(-1,2)

encoder = preprocessing.LabelEncoder()
df.loc[:, "aspect_tag"] = encoder.fit_transform(df["aspect_tag"])

sentences = df.groupby("num")["text"].apply(list).values
aspect_tags = df.groupby("num")["aspect_tag"].apply(list).values
polarity_tags = df.groupby("num")["polarity"].apply(list).values

polarity_unique_values = df.polarity.unique()

print('num of aspect tags: {}'.format(len(encoder.classes_)))
print('num of polarity tags: {}'.format(len(polarity_unique_values)))

np.where(encoder.classes_ == "AT")[0].item()

num of aspect tags: 2
num of polarity tags: 4


0

In [7]:
idx = 0
print(sentences[idx])
print(aspect_tags[idx])
print(polarity_tags[idx])

['I', 'charge', 'it', 'at', 'night', 'and', 'skip', 'taking', 'the', 'cord', 'with', 'me', 'because', 'of', 'the', 'good', 'battery', 'life']
[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]


---

In [8]:
def get_new_aspect_cluster(left, right, aspect_term, polarity, sentence, window_size=5):
    left = left[-window_size:]
    right = right[:window_size]

    if len(polarity) == 0:
        polarity = 0
    else:
        polarity = int(sum(polarity)/len(polarity))
   
    left.extend(aspect_term)
    left.extend(right)
    return {
        "local_context":left,
        "global_context":sentence,
        "aspect_term":aspect_term,
        "polarity":polarity,
    }

def chop(sentence, aspect_tag, polarity_tag, window_size=5):
    ret_aspect_clusters = []
#     ret_aspect_clusters = {
#         "context":list(),
#         "aspect_term":list(),
#         "polarity":list(),
#     }
    left = []
    right = []
    aspect_term = []
    polarity = []
    doing_left = True
    doing_right = False
    doing_aspect = False
    for i in range(len(sentence)):
        # check if the current token is an aspect term
        if aspect_tag[i] == 0:
            if doing_left:
                doing_aspect = True
                doing_left = False
            elif doing_right:
                doing_right = False
                doing_aspect = True
                # Now, need to save the previous aspect term cluster
                ret_aspect_clusters.append(get_new_aspect_cluster(
                    left, right, aspect_term, polarity, sentence, window_size=window_size))
                left = right
                right = []
                aspect_term = []
                polarity = []
            aspect_term.append(sentence[i])
            polarity.append(polarity_tag[i])
        else:
            if doing_left:
                left.append(sentence[i])
            elif doing_right:
                right.append(sentence[i])
            else:
                doing_aspect = False
                doing_right = True
                right.append(sentence[i])
                
    ret_aspect_clusters.append(get_new_aspect_cluster(
        left, right, aspect_term, polarity, sentence, window_size=window_size))
    
    return ret_aspect_clusters

In [9]:
def upsample_polarity(all_aspect_clusters):
    neg_list = []
    neu_list = []
    pos_list = []

    for item in all_aspect_clusters:
        if item['polarity'] == 2:
            neg_list.append(item)
        elif item['polarity'] == 0:
            neu_list.append(item)
        else:
            pos_list.append(item)

    num_to_add = len(pos_list) - len(neg_list)
    for i in range(num_to_add):
        idx = random.randint(0, len(neg_list)-1)
        neg_list.append(neg_list[idx])

    num_to_add = len(pos_list) - len(neu_list)
    for i in range(num_to_add):
        idx = random.randint(0, len(neu_list)-1)
        neu_list.append(neu_list[idx])    

    all_aspect_clusters = neg_list
    all_aspect_clusters.extend(neu_list)
    all_aspect_clusters.extend(pos_list)  

    return all_aspect_clusters

In [10]:
# generate word_index list
def build_vocab(df):
    word_idx = {}
    for idx, word in enumerate(sorted(set(df.text.values))):
        word_idx[word] = idx + 1
    return word_idx

def gen_np_embedding(fn, word_idx, dim=100, emb=False):
    if emb:
        model = load_model(fn + ".bin")
    embedding = np.zeros((len(word_idx) + 2, dim))

    with open(fn, encoding="utf8") as f:
        for l in f:
            # for each line, get the word and its vector
            rec = l.rstrip().split(' ')
            if len(rec) == 2:  # skip the first line.
                continue
                # if the word in word_idx, fill the embedding
            if rec[0] in word_idx:
                embedding[word_idx[rec[0]]] = np.array([float(r) for r in rec[1:]])
    for w in word_idx:
        if embedding[word_idx[w]].sum() == 0.:
            if emb:
                embedding[word_idx[w]] = model.get_word_vector(w)
    return embedding

def create_train_data_restaurant(all_aspect_clusters,  word_idx, sent_len=83):

    train_X_local = np.zeros((len(all_aspect_clusters), sent_len), np.int16)
    train_X_global = np.zeros((len(all_aspect_clusters), sent_len), np.int16)

    train_y = np.zeros(len(all_aspect_clusters), np.int16)

    # iterate the asoect
    for sx, sent in enumerate(all_aspect_clusters):
        train_y[sx] = sent['polarity']
        global_sentence = sent['global_context']
        local_sentence = sent['local_context']
        # write word index and tag in train_X
        try:
            for wx, word in enumerate(global_sentence):
                train_X_global[sx, wx] = word_idx[word]

            for wx, word in enumerate(local_sentence):
                train_X_local[sx, wx] = word_idx[word]

        except KeyError:
            continue

    return train_X_local, train_X_global, train_y


def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')


def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    elif isinstance(data, dict):
        for k, v in data.items():
            data[k] = v.to(device)
        return data
    return data.to(device, non_blocking=True)


class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""

    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)


def loss_fn(pred, label, num_label):
    pred = pred.view(-1,num_label)
    label = label.view(-1)
    loss = torch.nn.functional.cross_entropy(pred, label)
    return loss


def cal_acc(pred_tags, true_tags):
    if isinstance(pred_tags, list):
        pred_tags = torch.cat(pred_tags, 0)
        true_tags = torch.cat(true_tags, 0)
    acc = (pred_tags == true_tags).sum() / pred_tags.numel()
    f1 = f1_score(true_tags.cpu().numpy(), pred_tags.cpu().numpy(), labels=[0, 1, 2], average='weighted')
    cm = confusion_matrix(true_tags.cpu().numpy(), pred_tags.cpu().numpy())
    return acc, f1, cm


class Model(torch.nn.Module):
    def __init__(self, gen_emb, num_classes=3, alpha = 0.6):
        super(Model, self).__init__()
  
        self.embed = nn.Embedding.from_pretrained(torch.tensor(gen_emb, dtype=torch.float))

        self.lstm = nn.LSTM(gen_emb.shape[1], hidden_size=150, num_layers=1, bidirectional=True, batch_first=True)
        self.dense = nn.Linear(150*2, num_classes)

        self.alpha = alpha
        self.dropout = nn.Dropout(0.3)


    def forward(self, x_train_local, x_train_global):
        x_local = self.embed(x_train_local)
        
        local_seq_lengths = np.sum(np.array(x_train_local) !=0, axis=1)
   
        x_emb_l = torch.nn.utils.rnn.pack_padded_sequence(x_local, local_seq_lengths, batch_first=True, enforce_sorted=False)

        _, (h_n, _) = self.lstm(x_emb_l.float())

        h_n = torch.cat([h_n[-2,:,:], h_n[-1,:,:]], dim=1)

        x_global = self.embed(x_train_global)


        global_seq_lengths = np.sum(np.array(x_train_global) !=0, axis=1)

        x_emb_g = torch.nn.utils.rnn.pack_padded_sequence(x_global, global_seq_lengths, batch_first=True, enforce_sorted=False)

        _, (h_n_g,_) = self.lstm(x_emb_g.float())

        h_n_g = torch.cat([h_n_g[-2,:,:], h_n_g[-1,:,:]], dim=1)

        avg_pool = torch.div(torch.add(h_n * (1 - self.alpha), h_n_g * self.alpha), 2)
        out = self.dropout(avg_pool)

        out = self.dense(out)

        out = torch.nn.functional.log_softmax(out)

        return out

In [11]:
word_indx = build_vocab(df)
    
fn = DATA_DIR + 'glove.840B.300d.txt'
general_embedding = gen_np_embedding(fn, word_indx, dim=300, emb=False)

In [15]:
all_aspect_clusters = []
for i in range(len(sentences)):
    all_aspect_clusters.extend(chop(sentences[i], aspect_tags[i], polarity_tags[i]))

local_context, global_context, y = create_train_data_restaurant(all_aspect_clusters ,word_indx, sent_len=85)

X_l_train, X_l_valid, X_g_train, X_g_valid,  y_train, y_valid = train_test_split(local_context, global_context, y, test_size=VALID_SIZE)


In [16]:
device = get_default_device()

NUM_EPOCHS = 20
TRAIN_BATCH_SIZE = 128
VALID_BATCH_SIZE = 1024

NUM_POLARITY_TAGS = 3

history = {
    "train_loss": list(),
    "polarity_train_acc": list(),
    "valid_loss": list(),
    "polarity_valid_acc": list(),
}

best_loss = np.inf

dataset = TensorDataset(torch.Tensor(X_l_train), torch.Tensor(X_g_train), torch.Tensor(y_train))
print(f"train samples:{len(dataset)}")
train_loader = DataLoader(dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True, drop_last=True)

dataset_valid = TensorDataset(torch.Tensor(X_l_valid),  torch.Tensor(X_g_valid), torch.Tensor(y_valid))
print(f"valid samples:{len(dataset_valid)}")
test_loader = DataLoader(dataset_valid, batch_size=VALID_BATCH_SIZE)

model = to_device(Model(general_embedding,  num_classes=3, alpha = 0), device)

train samples:4641
valid samples:1161


In [17]:
print(model)

Model(
  (embed): Embedding(6395, 300)
  (lstm): LSTM(300, 150, batch_first=True, bidirectional=True)
  (dense): Linear(in_features=300, out_features=3, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)


In [13]:
def get_classification_report(test_loader, model, model_path=None):
    if model_path is not None: # load the saved model
        print('Loading saved model from: {}'.format(model_path))
        model.load_state_dict(torch.load(model_path))
    model = to_device(model, device)   
    
    model.eval()
    final_pred_polarity_tags = []
    final_true_polarity_tags = []


    with torch.no_grad():
        for data in tqdm(test_loader, total=len(test_loader)):
            for i in range(len(data)):
                data[i] = data[i].to(device)

            l_feature, g_feature, label = data
            l_feature, g_feature, label = l_feature.long(), g_feature.long(), label.long()
            pred_logits = model(l_feature, g_feature)

            pred_tags = pred_logits.max(-1)[1]

            final_pred_polarity_tags.extend(pred_tags)
            final_true_polarity_tags.extend(label)

    final_pred_polarity_tags = torch.stack(final_pred_polarity_tags).cpu()
    final_true_polarity_tags = torch.stack(final_true_polarity_tags).cpu()
        
    print(classification_report(final_true_polarity_tags, final_pred_polarity_tags, 
                                target_names=["Neutral", "Positive", "Negative"]))

## upsampling = False

In [16]:
lst = [0, 0.2, 0.4, 0.6, 0.8, 1]
lst = [0.6, 0.8, 1]

for alpha_val in lst:
    choice = [2, 4, 8]
    upsampling = False

    for window in choice:

        print()
        print("*"*90)
        print("Alpha Size: {}".format(alpha_val))
        print("Window Size: {}".format(window))
        print("*"*90)
        print()

        all_aspect_clusters = []
        for i in range(len(sentences)):
            all_aspect_clusters.extend(chop(sentences[i], aspect_tags[i], polarity_tags[i], 
                                            window_size=window))
        if upsampling:
            all_aspect_clusters = upsample_polarity(all_aspect_clusters)

        local_context, global_context, y = create_train_data_restaurant(all_aspect_clusters ,word_indx, sent_len=85)

        X_l_train, X_l_valid, X_g_train, X_g_valid,  y_train, y_valid = train_test_split(local_context, global_context, y, test_size=VALID_SIZE)

        device = get_default_device()

        NUM_EPOCHS = 20
        TRAIN_BATCH_SIZE = 128
        VALID_BATCH_SIZE = 1024

        NUM_POLARITY_TAGS = 3

        history = {
            "train_loss": list(),
            "polarity_train_acc": list(),
            "valid_loss": list(),
            "polarity_valid_acc": list(),
        }

        best_loss = np.inf

        dataset = TensorDataset(torch.Tensor(X_l_train), torch.Tensor(X_g_train), torch.Tensor(y_train))
        print(f"train samples:{len(dataset)}")
        train_loader = DataLoader(dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True, drop_last=True)

        dataset_valid = TensorDataset(torch.Tensor(X_l_valid),  torch.Tensor(X_g_valid), torch.Tensor(y_valid))
        print(f"valid samples:{len(dataset_valid)}")
        test_loader = DataLoader(dataset_valid, batch_size=VALID_BATCH_SIZE)

        model = to_device(Model(general_embedding,  num_classes=3, alpha = alpha_val), device)

        torch.cuda.empty_cache()

        parameters = [p for p in model.parameters() if p.requires_grad]
        optimizer = AdamW(parameters, lr=1e-4)


        for epoch in range(NUM_EPOCHS):
            train_losses = []
            train_acc = []
            test_loss = []
            test_acc = []
            train_f1 = []
            test_f1 = []

            model.train()
            preds = []
            masks = []
            labels = []
            for data in tqdm(train_loader, total=len(train_loader)):
                for i in range(len(data)):
                    data[i] = data[i].to(device)
                l_feature, g_feature, label = data
                l_feature, g_feature, label = l_feature.long(), g_feature.long(), label.long()

                optimizer.zero_grad()
                pred_logits = model(l_feature, g_feature)
                loss = loss_fn(pred_logits, label, 3)
                loss.backward()
                optimizer.step()

                train_losses.append(loss.item())

                pred_tags = pred_logits.max(-1)[1]
                preds.append(pred_tags)
                labels.append(label)

            avg_train_acc, avg_train_f1, train_cm = cal_acc(preds,labels)
            avg_train_loss = sum(train_losses) / len(train_losses)

            preds = []
            masks = []
            labels = []
            with torch.no_grad():
                for data in tqdm(test_loader, total=len(test_loader)):
                    for i in range(len(data)):
                        data[i] = data[i].to(device)
                    l_feature, g_feature, label = data
                    l_feature, g_feature, label = l_feature.long(), g_feature.long(), label.long()
                    pred_logits = model(l_feature, g_feature)
                    loss = loss_fn(pred_logits, label, 3)

                    test_loss.append(loss.item())

                    pred_tags = pred_logits.max(-1)[1]

                    preds.append(pred_tags)
                    labels.append(label)

            avg_test_acc, avg_test_f1, test_cm = cal_acc(preds,  labels)
            avg_test_loss = sum(test_loss) / len(test_loss)

            print(f"\nepoch {epoch}")
            print("\ttrain_loss:{:.3f} valid_loss:{:.3f}".format(avg_train_loss, avg_test_loss))
            print("\ttrain_acc:{:.2%} valid_acc:{:.2%}".format(avg_train_acc, avg_test_acc))
            print("\ttrain_f1:{:.3f} valid_f1:{:.3f}".format(avg_train_f1, avg_test_f1))
            print(f"\ttrain_confusion_matrix:\n{train_cm}")
            print(f"\tvalid_confusion_matrix:\n{test_cm}")

            if avg_test_loss < best_loss:
                torch.save(model.state_dict(), MODEL_PATH)
                best_loss = avg_test_loss    
                
            history['train_loss'].append(avg_train_loss)
            history['polarity_train_acc'].append(avg_train_acc.cpu().numpy())
            history['valid_loss'].append(avg_test_loss)
            history['polarity_valid_acc'].append(avg_test_acc.cpu().numpy())

        get_classification_report(test_loader, model, model_path=MODEL_PATH)
                
            



******************************************************************************************
Alpha Size: 0.6
Window Size: 2
******************************************************************************************

train samples:4641
valid samples:1161


  0%|          | 0/36 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_8755/1114866685.py:147: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = torch.nn.functional.log_softmax(out)
  0%|          | 0/2 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_8755/1114866685.py:147: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = torch.nn.functional.log_softmax(out)
100%|██████████| 2/2 [00:00<00:00,  4.21it/s]



epoch 0
	train_loss:1.064 valid_loss:1.034
	train_acc:51.39% valid_acc:53.40%
	train_f1:0.374 valid_f1:0.377
	train_confusion_matrix:
[[  41  850    0]
 [  90 2319    4]
 [  46 1250    8]]
	valid_confusion_matrix:
[[  0 220   0]
 [  0 616   3]
 [  0 318   4]]


  0%|          | 0/36 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_8755/1114866685.py:147: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = torch.nn.functional.log_softmax(out)
  0%|          | 0/2 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_8755/1114866685.py:147: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = torch.nn.functional.log_softmax(out)
100%|██████████| 2/2 [00:00<00:00,  4.19it/s]



epoch 1
	train_loss:1.005 valid_loss:0.988
	train_acc:52.39% valid_acc:53.49%
	train_f1:0.362 valid_f1:0.376
	train_confusion_matrix:
[[   0  898    0]
 [   0 2406    1]
 [   0 1295    8]]
	valid_confusion_matrix:
[[  0 220   0]
 [  0 618   1]
 [  0 319   3]]


...


epoch 18
	train_loss:0.631 valid_loss:0.704
	train_acc:74.65% valid_acc:72.44%
	train_f1:0.728 valid_f1:0.705
	train_confusion_matrix:
[[ 266  312  316]
 [  80 2138  197]
 [ 107  156 1036]]
	valid_confusion_matrix:
[[ 59  80  83]
 [ 27 515  68]
 [ 15  47 267]]


  0%|          | 0/36 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_8755/1114866685.py:147: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = torch.nn.functional.log_softmax(out)
  0%|          | 0/2 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_8755/1114866685.py:147: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = torch.nn.functional.log_softmax(out)
100%|██████████| 2/2 [00:00<00:00,  3.64it/s]



epoch 19
	train_loss:0.629 valid_loss:0.702
	train_acc:74.57% valid_acc:70.71%
	train_f1:0.729 valid_f1:0.693
	train_confusion_matrix:
[[ 274  290  325]
 [  96 2130  191]
 [ 105  165 1032]]
	valid_confusion_matrix:
[[ 66  92  64]
 [ 42 517  51]
 [ 37  54 238]]
Loading saved model from: model_task2.bin


  0%|          | 0/2 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_8755/1114866685.py:147: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = torch.nn.functional.log_softmax(out)
100%|██████████| 2/2 [00:00<00:00,  4.07it/s]

              precision    recall  f1-score   support

     Neutral       0.54      0.27      0.36       222
    Positive       0.78      0.87      0.82       610
    Negative       0.66      0.74      0.70       329

    accuracy                           0.72      1161
   macro avg       0.66      0.63      0.63      1161
weighted avg       0.70      0.72      0.70      1161



## upsampling = True

In [18]:
import random
lst = [0, 0.2, 0.4]
#[0.6, 0.8, 1]

for alpha_val in lst:
    choice = [2, 4, 8]
    upsampling = True

    for window in choice:

        print()
        print("*"*90)
        print("Alpha Size: {}".format(alpha_val))
        print("Window Size: {}".format(window))
        print("*"*90)
        print()

        all_aspect_clusters = []
        for i in range(len(sentences)):
            all_aspect_clusters.extend(chop(sentences[i], aspect_tags[i], polarity_tags[i], 
                                            window_size=window))
        if upsampling:
            all_aspect_clusters = upsample_polarity(all_aspect_clusters)

        local_context, global_context, y = create_train_data_restaurant(all_aspect_clusters ,word_indx, sent_len=85)

        X_l_train, X_l_valid, X_g_train, X_g_valid,  y_train, y_valid = train_test_split(local_context, global_context, y, test_size=VALID_SIZE)

        device = get_default_device()

        NUM_EPOCHS = 20
        TRAIN_BATCH_SIZE = 128
        VALID_BATCH_SIZE = 1024

        NUM_POLARITY_TAGS = 3

        history = {
            "train_loss": list(),
            "polarity_train_acc": list(),
            "valid_loss": list(),
            "polarity_valid_acc": list(),
        }

        best_loss = np.inf

        dataset = TensorDataset(torch.Tensor(X_l_train), torch.Tensor(X_g_train), torch.Tensor(y_train))
        print(f"train samples:{len(dataset)}")
        train_loader = DataLoader(dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True, drop_last=True)

        dataset_valid = TensorDataset(torch.Tensor(X_l_valid),  torch.Tensor(X_g_valid), torch.Tensor(y_valid))
        print(f"valid samples:{len(dataset_valid)}")
        test_loader = DataLoader(dataset_valid, batch_size=VALID_BATCH_SIZE)

        model = to_device(Model(general_embedding,  num_classes=3, alpha = alpha_val), device)

        torch.cuda.empty_cache()

        parameters = [p for p in model.parameters() if p.requires_grad]
        optimizer = AdamW(parameters, lr=1e-4)


        for epoch in range(NUM_EPOCHS):
            train_losses = []
            train_acc = []
            test_loss = []
            test_acc = []
            train_f1 = []
            test_f1 = []

            model.train()
            preds = []
            masks = []
            labels = []
            for data in tqdm(train_loader, total=len(train_loader)):
                for i in range(len(data)):
                    data[i] = data[i].to(device)
                l_feature, g_feature, label = data
                l_feature, g_feature, label = l_feature.long(), g_feature.long(), label.long()

                optimizer.zero_grad()
                pred_logits = model(l_feature, g_feature)
                loss = loss_fn(pred_logits, label, 3)
                loss.backward()
                optimizer.step()

                train_losses.append(loss.item())

                pred_tags = pred_logits.max(-1)[1]
                preds.append(pred_tags)
                labels.append(label)

            avg_train_acc, avg_train_f1, train_cm = cal_acc(preds,labels)
            avg_train_loss = sum(train_losses) / len(train_losses)

            preds = []
            masks = []
            labels = []
            with torch.no_grad():
                for data in tqdm(test_loader, total=len(test_loader)):
                    for i in range(len(data)):
                        data[i] = data[i].to(device)
                    l_feature, g_feature, label = data
                    l_feature, g_feature, label = l_feature.long(), g_feature.long(), label.long()
                    pred_logits = model(l_feature, g_feature)
                    loss = loss_fn(pred_logits, label, 3)

                    test_loss.append(loss.item())

                    pred_tags = pred_logits.max(-1)[1]

                    preds.append(pred_tags)
                    labels.append(label)

            avg_test_acc, avg_test_f1, test_cm = cal_acc(preds,  labels)
            avg_test_loss = sum(test_loss) / len(test_loss)

            print(f"\nepoch {epoch}")
            print("\ttrain_loss:{:.3f} valid_loss:{:.3f}".format(avg_train_loss, avg_test_loss))
            print("\ttrain_acc:{:.2%} valid_acc:{:.2%}".format(avg_train_acc, avg_test_acc))
            print("\ttrain_f1:{:.3f} valid_f1:{:.3f}".format(avg_train_f1, avg_test_f1))
            print(f"\ttrain_confusion_matrix:\n{train_cm}")
            print(f"\tvalid_confusion_matrix:\n{test_cm}")

            if avg_test_loss < best_loss:
                torch.save(model.state_dict(), MODEL_PATH)
                best_loss = avg_test_loss    
                
            history['train_loss'].append(avg_train_loss)
            history['polarity_train_acc'].append(avg_train_acc.cpu().numpy())
            history['valid_loss'].append(avg_test_loss)
            history['polarity_valid_acc'].append(avg_test_acc.cpu().numpy())

        get_classification_report(test_loader, model, model_path=MODEL_PATH)
                
            



******************************************************************************************
Alpha Size: 0
Window Size: 2
******************************************************************************************

train samples:7305
valid samples:1827


  0%|          | 0/57 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_8755/1114866685.py:147: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = torch.nn.functional.log_softmax(out)
  0%|          | 0/2 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_8755/1114866685.py:147: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = torch.nn.functional.log_softmax(out)
100%|██████████| 2/2 [00:00<00:00,  4.09it/s]



epoch 0
	train_loss:1.088 valid_loss:1.075
	train_acc:44.59% valid_acc:51.89%
	train_f1:0.442 valid_f1:0.507
	train_confusion_matrix:
[[ 932  319 1166]
 [ 706  867  867]
 [ 686  299 1454]]
	valid_confusion_matrix:
[[195 161 269]
 [ 70 374 155]
 [ 89 135 379]]


  0%|          | 0/57 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_8755/1114866685.py:147: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = torch.nn.functional.log_softmax(out)
  0%|          | 0/2 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_8755/1114866685.py:147: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = torch.nn.functional.log_softmax(out)
100%|██████████| 2/2 [00:00<00:00,  4.21it/s]



epoch 1
	train_loss:1.055 valid_loss:1.028
	train_acc:54.03% valid_acc:51.40%
	train_f1:0.526 valid_f1:0.484
	train_confusion_matrix:
[[ 747  736  933]
 [ 252 1676  512]
 [ 290  631 1519]]
	valid_confusion_matrix:
[[135 236 254]
 [ 38 456 105]
 [ 54 201 348]]


...


epoch 18
	train_loss:0.588 valid_loss:0.689
	train_acc:76.77% valid_acc:71.10%
	train_f1:0.768 valid_f1:0.710
	train_confusion_matrix:
[[1722  275  442]
 [ 305 1928  192]
 [ 340  141 1951]]
	valid_confusion_matrix:
[[364  71 166]
 [ 78 452  87]
 [ 83  43 483]]


  0%|          | 0/57 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_8755/1114866685.py:147: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = torch.nn.functional.log_softmax(out)
  0%|          | 0/2 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_8755/1114866685.py:147: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = torch.nn.functional.log_softmax(out)
100%|██████████| 2/2 [00:00<00:00,  4.28it/s]



epoch 19
	train_loss:0.569 valid_loss:0.670
	train_acc:77.66% valid_acc:71.54%
	train_f1:0.777 valid_f1:0.715
	train_confusion_matrix:
[[1766  253  420]
 [ 325 1914  183]
 [ 331  118 1986]]
	valid_confusion_matrix:
[[398  84 119]
 [ 89 474  54]
 [116  58 435]]
Loading saved model from: model_task2.bin


  0%|          | 0/2 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_8755/1114866685.py:147: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = torch.nn.functional.log_softmax(out)
100%|██████████| 2/2 [00:00<00:00,  4.34it/s]

              precision    recall  f1-score   support

     Neutral       0.66      0.67      0.67       601
    Positive       0.77      0.77      0.77       617
    Negative       0.72      0.72      0.72       609

    accuracy                           0.72      1827
   macro avg       0.72      0.72      0.72      1827
weighted avg       0.72      0.72      0.72      1827



In [19]:
import random
lst = [0.6, 0.8, 1]

for alpha_val in lst:
    choice = [2, 4, 8]
    upsampling = True

    for window in choice:

        print()
        print("*"*90)
        print("Alpha Size: {}".format(alpha_val))
        print("Window Size: {}".format(window))
        print("*"*90)
        print()

        all_aspect_clusters = []
        for i in range(len(sentences)):
            all_aspect_clusters.extend(chop(sentences[i], aspect_tags[i], polarity_tags[i], 
                                            window_size=window))
        if upsampling:
            all_aspect_clusters = upsample_polarity(all_aspect_clusters)

        local_context, global_context, y = create_train_data_restaurant(all_aspect_clusters ,word_indx, sent_len=85)

        X_l_train, X_l_valid, X_g_train, X_g_valid,  y_train, y_valid = train_test_split(local_context, global_context, y, test_size=VALID_SIZE)

        device = get_default_device()

        NUM_EPOCHS = 20
        TRAIN_BATCH_SIZE = 128
        VALID_BATCH_SIZE = 1024

        NUM_POLARITY_TAGS = 3

        history = {
            "train_loss": list(),
            "polarity_train_acc": list(),
            "valid_loss": list(),
            "polarity_valid_acc": list(),
        }

        best_loss = np.inf

        dataset = TensorDataset(torch.Tensor(X_l_train), torch.Tensor(X_g_train), torch.Tensor(y_train))
        print(f"train samples:{len(dataset)}")
        train_loader = DataLoader(dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True, drop_last=True)

        dataset_valid = TensorDataset(torch.Tensor(X_l_valid),  torch.Tensor(X_g_valid), torch.Tensor(y_valid))
        print(f"valid samples:{len(dataset_valid)}")
        test_loader = DataLoader(dataset_valid, batch_size=VALID_BATCH_SIZE)

        model = to_device(Model(general_embedding,  num_classes=3, alpha = alpha_val), device)

        torch.cuda.empty_cache()

        parameters = [p for p in model.parameters() if p.requires_grad]
        optimizer = AdamW(parameters, lr=1e-4)


        for epoch in range(NUM_EPOCHS):
            train_losses = []
            train_acc = []
            test_loss = []
            test_acc = []
            train_f1 = []
            test_f1 = []

            model.train()
            preds = []
            masks = []
            labels = []
            for data in tqdm(train_loader, total=len(train_loader)):
                for i in range(len(data)):
                    data[i] = data[i].to(device)
                l_feature, g_feature, label = data
                l_feature, g_feature, label = l_feature.long(), g_feature.long(), label.long()

                optimizer.zero_grad()
                pred_logits = model(l_feature, g_feature)
                loss = loss_fn(pred_logits, label, 3)
                loss.backward()
                optimizer.step()

                train_losses.append(loss.item())

                pred_tags = pred_logits.max(-1)[1]
                preds.append(pred_tags)
                labels.append(label)

            avg_train_acc, avg_train_f1, train_cm = cal_acc(preds,labels)
            avg_train_loss = sum(train_losses) / len(train_losses)

            preds = []
            masks = []
            labels = []
            with torch.no_grad():
                for data in tqdm(test_loader, total=len(test_loader)):
                    for i in range(len(data)):
                        data[i] = data[i].to(device)
                    l_feature, g_feature, label = data
                    l_feature, g_feature, label = l_feature.long(), g_feature.long(), label.long()
                    pred_logits = model(l_feature, g_feature)
                    loss = loss_fn(pred_logits, label, 3)

                    test_loss.append(loss.item())

                    pred_tags = pred_logits.max(-1)[1]

                    preds.append(pred_tags)
                    labels.append(label)

            avg_test_acc, avg_test_f1, test_cm = cal_acc(preds,  labels)
            avg_test_loss = sum(test_loss) / len(test_loss)

            print(f"\nepoch {epoch}")
            print("\ttrain_loss:{:.3f} valid_loss:{:.3f}".format(avg_train_loss, avg_test_loss))
            print("\ttrain_acc:{:.2%} valid_acc:{:.2%}".format(avg_train_acc, avg_test_acc))
            print("\ttrain_f1:{:.3f} valid_f1:{:.3f}".format(avg_train_f1, avg_test_f1))
            print(f"\ttrain_confusion_matrix:\n{train_cm}")
            print(f"\tvalid_confusion_matrix:\n{test_cm}")

            if avg_test_loss < best_loss:
                torch.save(model.state_dict(), MODEL_PATH)
                best_loss = avg_test_loss    
                
            history['train_loss'].append(avg_train_loss)
            history['polarity_train_acc'].append(avg_train_acc.cpu().numpy())
            history['valid_loss'].append(avg_test_loss)
            history['polarity_valid_acc'].append(avg_test_acc.cpu().numpy())

        get_classification_report(test_loader, model, model_path=MODEL_PATH)
                
            



******************************************************************************************
Alpha Size: 0.6
Window Size: 2
******************************************************************************************

train samples:7305
valid samples:1827


  0%|          | 0/57 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_8755/1114866685.py:147: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = torch.nn.functional.log_softmax(out)
  0%|          | 0/2 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_8755/1114866685.py:147: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = torch.nn.functional.log_softmax(out)
100%|██████████| 2/2 [00:00<00:00,  4.21it/s]



epoch 0
	train_loss:1.086 valid_loss:1.071
	train_acc:47.52% valid_acc:54.19%
	train_f1:0.456 valid_f1:0.535
	train_confusion_matrix:
[[ 563  707 1154]
 [ 295 1313  833]
 [ 264  576 1591]]
	valid_confusion_matrix:
[[234 168 215]
 [ 96 377 126]
 [107 125 379]]


  0%|          | 0/57 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_8755/1114866685.py:147: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = torch.nn.functional.log_softmax(out)
  0%|          | 0/2 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_8755/1114866685.py:147: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = torch.nn.functional.log_softmax(out)
100%|██████████| 2/2 [00:00<00:00,  4.40it/s]



epoch 1
	train_loss:1.044 valid_loss:0.990
	train_acc:54.99% valid_acc:55.88%
	train_f1:0.546 valid_f1:0.535
	train_confusion_matrix:
[[1013  592  820]
 [ 415 1551  476]
 [ 454  527 1448]]
	valid_confusion_matrix:
[[162 169 286]
 [ 39 414 146]
 [ 39 127 445]]


...


epoch 18
	train_loss:0.587 valid_loss:0.706
	train_acc:76.34% valid_acc:71.21%
	train_f1:0.763 valid_f1:0.712
	train_confusion_matrix:
[[1677  276  489]
 [ 297 1956  182]
 [ 322  160 1937]]
	valid_confusion_matrix:
[[367  55 175]
 [ 85 440  82]
 [ 86  43 494]]


  0%|          | 0/57 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_8755/1114866685.py:147: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = torch.nn.functional.log_softmax(out)
  0%|          | 0/2 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_8755/1114866685.py:147: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = torch.nn.functional.log_softmax(out)
100%|██████████| 2/2 [00:00<00:00,  4.13it/s]



epoch 19
	train_loss:0.575 valid_loss:0.699
	train_acc:77.26% valid_acc:72.52%
	train_f1:0.772 valid_f1:0.726
	train_confusion_matrix:
[[1728  260  457]
 [ 282 1974  179]
 [ 313  168 1935]]
	valid_confusion_matrix:
[[413  44 140]
 [112 428  67]
 [ 98  41 484]]
Loading saved model from: model_task2.bin


  0%|          | 0/2 [00:00<?, ?it/s]/var/folders/3c/71b8pg6d7j5dht_5h_2n4n_c0000gn/T/ipykernel_8755/1114866685.py:147: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = torch.nn.functional.log_softmax(out)
100%|██████████| 2/2 [00:00<00:00,  4.18it/s]

              precision    recall  f1-score   support

     Neutral       0.67      0.69      0.68       597
    Positive       0.83      0.71      0.77       607
    Negative       0.69      0.77      0.73       623

    accuracy                           0.72      1827
   macro avg       0.73      0.72      0.72      1827
weighted avg       0.73      0.72      0.72      1827



In [ ]:
import matplotlib.pyplot as plt
plt.plot(history['polarity_train_acc'], label='train accuracy')
plt.plot(history['polarity_valid_acc'], label='validation accuracy')
plt.title('Training history')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.ylim([0.4, 0.9])